# My Little Interview Prep Book [Algorithms]

**Stuff mentioned in this book: Data Structures and Algorithms**

In [14]:
# Just flushing out buffer asap.
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

## Stacks and Queues

**Stack of Plates** : Imagine a stack of plates. If the stack gets too high, it might topple. We would likely start a new stack when the previous stack exceeds a threshold. Implement data struct SetOfStacks that mimics this.

- _Composed of several stacks_

- _New stack once previous exceeds capacity_

In [15]:
class Slice:
    def __init__(self, data):
        self.data = data
        self.next = None

class MultiStack:
    def __init__(self, capacity):
        self.stack_size = 0
        self.stack_capacity = capacity
        self.cur_index = 0
        self.stack = []
    def push(self, data):
        ''' General guidelines for push:
        When inserting into end of stack or start of the entire stack class:
            create new stack and make that the initial Slice.
        When inserting normally:
            create new Slice and make the new Slice's next to previous head. Set new Slice as head.            
        '''
        if self.cur_index == self.stack_capacity or self.stack_size == 0:
            self.stack.append(Slice(data))
            self.stack_size+=1
            self.cur_index = 1
        else:
            s = Slice(data)
            s.next = self.stack[self.stack_size-1]
            self.stack[self.stack_size-1] = s
            self.cur_index += 1
            
    def pop(self):
        ''' General guidelines for pop:
        When popping beginning of stack:
            get value of the beginning of stack(when there's 1 element), and then set it to None and delete it
        When popping normally:
            get value of Slice and make the Slice equal to the next Slice
        '''
        # IF there are no more elements left:
        if self.stack_size == 0:
            return None
        
        rtVal = self.stack[self.stack_size-1].data
        if self.cur_index == 1:
            self.stack[self.stack_size-1] = None
            del self.stack[self.stack_size-1]
            self.cur_index = self.stack_capacity
            self.stack_size -= 1
            # in this case, cur_index goes to 0. Let's make
            return rtVal
        else:
            self.stack[self.stack_size-1] = self.stack[self.stack_size-1].next
            self.cur_index -= 1
            return rtVal

In [48]:
stack = MultiStack(2)
stack.push(4)
stack.push(2)
stack.push(1)
stack.push(6)

print(stack.stack[0].data)
print(stack.stack[1].next.data)
print(stack.stack_size)

print("---")

print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())

stack.push(4)
stack.push(2)
stack.push(1)
stack.push(6)

print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())

2
1
2
---
6
1
2
4
None
6
1
2
4
None


**Queue via stacks**: create a DoubleStackQueue class which implements a queue using two stacks

In [44]:
class DoubleStackQueue:
    def __init__(self):
        self.s1 = []
        self.s2 = []
    def front(self):
        ''' General Guidelines for front()
        front should take the s1 and flip it onto s2. Now the first item is in the front. pop s2's first
        stack frame and then flip the rest into s1.
        '''
        self.flip(self.s2, self.s1)
        retVal = self.s2.pop()
        return retVal
    def push(self, val):
        self.flip(self.s1, self.s2)
        self.s1.append(val)
    ''' IMPLEMENTATION CHANGE: GAYLE LAAKMANN MCDOWELL SUGGESTS TO FLIP ONLY WHEN ALTERNATING. 
    This causes the amortized speed to go down an order of magnitude.
    '''
    def flip(self, s1, s2):
        ''' flips s2 to s1 '''
        while len(s2) != 0:
            s1.append(s2.pop())

In [45]:
dsq = DoubleStackQueue()
dsq.push(5)
dsq.push(6)
dsq.push(2)
dsq.push(10)

print(dsq.front())
print(dsq.front())
print(dsq.front())
print(dsq.front())

5
6
2
10


**Sort Stack**: Write a program to sort a stack such that the smallest items are on the top. You can use an additional temporary stack, but you may not copy the elements into ANY OTHER DATA STRUCTURE(array for e.x.). I can only use push(), pop(), peek() and isEmpty().

In [46]:
def sort_stack(stack):
    ''' General Guidelines for sort_stack()
    Given the stack, unload it into s1.
    Traverse through the stack once and find the maximum value.
    We now know the index, pop it until you get to the max value. 
    Pop max value onto s3
    Pop rest of the values onto s2
    
    Quick question: What's the O() of this algorithm?
    We can see that worst case while loop = O(N)
    We can see inside the while loop = O(N + N-1 + N-2 + N-3 ... + 1) = O(N/2)
    We can see the reconstruction while loop = O(N)
    Therefore it is O(N * (N/2 + N)) = O(N^2) for this sort.
    '''
    # if the stack has a size of 1 or 0, it's already sorted!
    if len(stack) < 2:
        return stack
    s1 = []
    s2 = []
    count = 0 # so far sorted 0 elements
    
    while count != len(stack)-1 : # while not all of the elements are sorted
        # set the first val as the "max" for now. We need to find the actual one though:
        maxVal = None
        # this while loop gets the maximum from the stack
        while len(stack) != count:
            curVal = stack.pop()
            if maxVal is None or maxVal < curVal:
                # check if there is a value in the s2 stack
                if len(s2) != 0:
                    # pop the value into s1, because we know it's less than curVal
                    s1.append(s2.pop())
                # we will put the special "max" value in s2.
                s2.append(curVal)
                maxVal = curVal
            else:
                s1.append(curVal)
        
        # now reconstruct the stack with the bottom 
        stack.append(s2.pop())
        while len(s1) != 0:
            stack.append(s1.pop())
        count += 1        

In [47]:
stack = [5, 2, 3, 5, 6, 7, 1, 1, 100, 8, 10]

sort_stack(stack)

print(stack)

[100, 10, 8, 7, 6, 5, 5, 3, 2, 1, 1]


**Animal Shelter** : An animal shelter, which holds only dogs and cats, operates on a strictly "first in first out" basis. Adopt either "oldest" or oldest dog or cat. 

In [76]:
'''General Guidelines for Dog and Cat
Dogs and cats are simply linkedlist nodes that points to the next dog or cat
'''
class Animal:
    def __init__(self, name):
        self.name = name
        self.next = None
    def get_animal_type(self):
        pass
    def get_name(self):
        return self.name
class Dog(Animal):
    def __init__(self, name):
        Animal.__init__(self, name)
    def get_animal_type(self):
        return "Dog"
class Cat(Animal):
    def __init__(self, name):
        Animal.__init__(self, name)
    def get_animal_type(self):
        return "Cat"
    

In [100]:
import numpy as np

class AnimalShelter:
    '''General Guidelines for AnimalShelter
    Is basically a queue consisting of Dogs and Cats. We implement the queue with a circular array.
    Make it hard for ourselves. Python is spoiling us of our CS skillz so let's import fixed sized array
    from numpy.
    '''
    def __init__(self, capacity):
        self.q = np.empty((capacity), dtype=object)
        self.capacity = capacity
        self.oldestCat = self.oldestDog = None
        self.head = self.size = self.tail = 0
    def iter_til_newest_animal(self, node):
        while node is not None and node.next is not None:
            node = node.next
        return node
    def push(self, animal):
        ''' General Guidelines for push()
        Do the normal push() stuff
        Make sure when you add an animal, you link it to the previous Dog or the previous Cat
        '''
        if self.tail == self.head and self.size != 0:
            return False
        self.q[self.tail] = animal
        
        if animal.get_animal_type() == "Cat":
            newCat = self.iter_til_newest_animal(self.oldestCat)
            if newCat is None:
                self.oldestCat = animal
            else:
                newCat.next = animal
        else:
            newDog = self.iter_til_newest_animal(self.oldestDog)
            if newDog is None:
                self.oldestDog = animal
            else:
                newDog.next = animal
            
        self.tail = (self.tail + 1) % self.capacity
        self.size += 1
        return True
    def readjust(self, retVal):
        temp = None
        for i in range(self.size):
            curIndex = (i+self.head) % self.capacity
            nextIndex = (i+self.head+1) % self.capacity
            temp = self.q[nextIndex]
            self.q[nextIndex] = self.q[curIndex]
            if retVal == temp:
                break            
    def pop(self):
        if self.size == 0:
            return None
        retVal = self.q[self.head]
        self.head = (self.head + 1) % self.capacity
        self.size -= 1
        return retVal
    def popDog(self):
        if self.size == 0 or self.oldestDog is None:
            return None
        retVal = self.oldestDog
        self.oldestDog = self.oldestDog.next
        # Now we possibly screwed up the queue, so let's readjust the head up one.
        self.readjust(retVal)
        return retVal
    def popCat(self):
        if self.size == 0 or self.oldestDog is None:
            return None
        retVal = self.oldestCat
        self.oldestCat = self.oldestCat.next
        # Now we possibly screwed up the queue, so let's readjust the head up one.
        self.readjust(retVal)
        return retVal
        

In [101]:
d1 = Dog("Woofy")
d2 = Dog("Sparky")

c1 = Cat("Whiskers")
c2 = Cat("Cinnamon")

print(d1.get_animal_type())
print(d1.get_name())
print("---")
a_s = AnimalShelter(5)
a_s.push(d1)
a_s.push(c1)
a_s.push(d2)
a_s.push(c2)
print(a_s.popDog().get_name())
print(a_s.popDog().get_name())


Dog
Woofy
---
Woofy
Sparky
